In [ ]:
#downloading dependencies
%pip install groq -q -q -q --exists-action i
%pip install -U langchain-community -q -q -q --exists-action i
%pip install -U langchain-huggingface -q -q -q --exists-action i
%pip install faiss-cpu -q -q -q --exists-action i
%pip install streamlit -q -q -q --exists-action i
%pip install sentence-transformers -q -q -q --exists-action i


In [ ]:
#importing required packages
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import time
import json
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from groq import Groq
import streamlit as st


In [ ]:
#confirming output directory path

output_dir = "RAG Project"
os.makedirs(output_dir, exist_ok=True)
print(f"Directory '{output_dir}' ensured to exist.")

Directory 'C:\Users\pooja\OneDrive\Desktop\Documents\RAG Project' ensured to exist.


In [ ]:
#scraping and storing IPC related data

BASE_URL = "https://www.indiacode.nic.in/show-data?actid=AC_CEN_5_23_00023_186045_1523266427465&sectionId={sid}&sectionno={secno}"

def extract_ipc_section(sec_no):
    url = BASE_URL.format(sid=sec_no, secno=sec_no)
    response = requests.get(url)
    if "No Data Found" in response.text:
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    text = soup.get_text(separator="\n")
    lines = text.splitlines()
    lines = [line.strip() for line in lines if line.strip()]

    title = lines[0] if lines else f"Section {sec_no}"
    content = "\n".join(lines[1:]) if len(lines) > 1 else ""

    return {
        "section_no": str(sec_no),
        "title": title,
        "ipc_text": content,
        "source": url,
        "law_code": "IPC",
        "valid_from": "1860-01-01"
    }

def scrape_ipc(start=1, end=511):
    all_sections = []
    for sec_no in tqdm(range(start, end + 1)):
        try:
            data = extract_ipc_section(sec_no)
            if data:
                all_sections.append(data)
                print(f"Section {sec_no} extracted.")
            else:
                print(f"Section {sec_no} not found.")
            time.sleep(0.3) 
        except Exception as e:
            print(f"Error in Section {sec_no}: {e}")
            continue
    return all_sections

ipc_data = scrape_ipc()
with open("RAG Project\\ipc_sections.jsonl", "w", encoding="utf-8") as f:
    for item in ipc_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"\nScraped {len(ipc_data)} IPC sections and saved to ipc_sections.jsonl")

  0%|          | 0/511 [00:00<?, ?it/s]

Section 1 extracted.


  0%|          | 1/511 [00:00<05:32,  1.53it/s]

Section 2 extracted.


  0%|          | 2/511 [00:01<05:51,  1.45it/s]

...

Section 510 extracted.


100%|█████████▉| 510/511 [04:09<00:00,  2.14it/s]

Section 511 extracted.


100%|██████████| 511/511 [04:10<00:00,  2.04it/s]


Scraped 511 IPC sections and saved to ipc_sections.jsonl


In [ ]:
#scraping and storing BNS related data

BASE_URL = "https://www.indiacode.nic.in/show-data?actid=AC_CEN_1_123_00001_2023_50001&sectionId={sid}&sectionno={clause}"

def extract_bns_clause(clause_no):
    url = BASE_URL.format(sid=clause_no, clause=clause_no)
    response = requests.get(url)
    if "No Data Found" in response.text:
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    text = soup.get_text(separator="\n")
    lines = text.splitlines()
    lines = [line.strip() for line in lines if line.strip()]

    title = lines[0] if lines else f"Clause {clause_no}"
    content = "\n".join(lines[1:]) if len(lines) > 1 else ""

    return {
        "clause_no": str(clause_no),
        "title": title,
        "bns_text": content,
        "source": url,
        "law_code": "BNS",
        "valid_from": "2025-07-01"
    }

def scrape_bns(start=1, end=358):
    all_clauses = []
    for clause_no in tqdm(range(start, end + 1)):
        try:
            clause = extract_bns_clause(clause_no)
            if clause:
                all_clauses.append(clause)
                print(f"Clause {clause_no} extracted.")
            else:
                print(f"Clause {clause_no} not found.")
            time.sleep(0.3)
        except Exception as e:
            print(f"Error in Clause {clause_no}: {e}")
    return all_clauses


bns_data = scrape_bns()
path = "RAG Project\\bns_clauses.jsonl"
with open(path, "w", encoding="utf-8") as f:
    for item in bns_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")
print(f"\nSaved {len(bns_data)} BNS clauses to {path}")


  0%|          | 0/358 [00:00<?, ?it/s]

Clause 1 extracted.


  0%|          | 1/358 [00:00<03:10,  1.87it/s]

Clause 2 extracted.


  1%|          | 2/358 [00:00<02:48,  2.11it/s]

...

Clause 357 extracted.


100%|█████████▉| 357/358 [02:57<00:00,  2.13it/s]

Clause 358 extracted.


100%|██████████| 358/358 [02:58<00:00,  2.01it/s]


Saved 358 BNS clauses to C:\Users\pooja\OneDrive\Desktop\Documents\RAG Project\bns_clauses.jsonl


In [ ]:
#scraping IPC-BNS mapping

path = "RAG Project\\COMPARISON SUMMARY BNS to IPC.xlsx"
tables = pd.read_excel(path)

ipc_to_bns = {}
for _, row in tables.iterrows():
    try:
        ipc = str(row[2]).strip()
        bns = str(row[0]).strip()
        if ipc and bns:
            ipc_to_bns[ipc] = bns
    except Exception as e:
        print(f"Skipping row due to error: {e}")

print(f"Extracted {len(ipc_to_bns)} mappings")

Extracted 513 mappings


C:\Users\pooja\AppData\Local\Temp\ipykernel_9872\653162041.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ipc = str(row[2]).strip()
C:\Users\pooja\AppData\Local\Temp\ipykernel_9872\653162041.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  bns = str(row[0]).strip()


In [ ]:
#creating a corpus with both the data

def load_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]

ipc_data = load_jsonl("RAG Project\\ipc_sections.jsonl")
bns_data = load_jsonl("RAG Project\\bns_clauses.jsonl")


bns_lookup = {clause["clause_no"]: clause for clause in bns_data}


dual_tagged = []
for ipc in ipc_data:
    sec = ipc["section_no"]
    bns_clause = ipc_to_bns.get(sec)

    merged = {
        "ipc_section": sec,
        "bns_clause": bns_clause,
        "title": ipc["title"],
        "ipc_text": ipc["ipc_text"],
        "bns_text": bns_lookup.get(bns_clause, {}).get("bns_text") if bns_clause else None,
        "law_code": ["IPC"] if not bns_clause else ["IPC", "BNS"],
        "valid_from": {
            "IPC": "1860-01-01",
            "BNS": "2025-07-01" if bns_clause else None
        },
        "source": [
            ipc["source"],
            bns_lookup.get(bns_clause, {}).get("source") if bns_clause else None
        ]
    }

    dual_tagged.append(merged)


with open("RAG Project\\ipc_bns_dual_tagged.jsonl", "w", encoding="utf-8") as f:
    for item in dual_tagged:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"Dual-tagged corpus saved: {len(dual_tagged)} entries.")

Dual-tagged corpus saved: 511 entries.


In [ ]:
#Chunking and embedding corpus

DATA_PATH = "RAG Project\\ipc_bns_dual_tagged.jsonl"
VECTOR_DIR = "RAG Project\\vectorstore\\ipc_bns_law"
CHUNK_SIZE = 512
CHUNK_OVERLAP = 50

docs = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        ipc_section = item.get('ipc_section')
        bns_clause = item.get('bns_clause')
        title = item.get('title', '')
        ipc_text = item.get('ipc_text', '')
        bns_text = item.get('bns_text', '')
        source_ipc = item.get('source', [None, None])[0] 
        source_bns = item.get('source', [None, None])[1] 


        content = f"[IPC Section {ipc_section}]: {title}\n{ipc_text}\n"
        if bns_clause:
            content += f"[BNS Clause {bns_clause}]: {title}\n{bns_text}"


        metadata = {
            "ipc_section": ipc_section,
            "bns_clause": bns_clause,
            "title": title,
            "source_ipc": source_ipc,
            "source_bns": source_bns,
            "type": "law"
        }
        docs.append(Document(page_content=content, metadata=metadata))

print(f"Loaded {len(docs)} dual-tagged documents")

splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)
chunked_docs = splitter.split_documents(docs)
print(f"Chunked into {len(chunked_docs)} total chunks")

embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(chunked_docs, embed_model)
os.makedirs(VECTOR_DIR, exist_ok=True)
vectorstore.save_local(VECTOR_DIR)
print(f"Vector store saved to '{VECTOR_DIR}'")

In [ ]:
#prompting 
client = Groq(api_key="")#enter your API key

VECTOR_DIR = "RAG Project\\vectorstore\\ipc_bns_law"
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local(VECTOR_DIR, embedding_model, allow_dangerous_deserialization=True)


def query(ques):
  ques = ques
  retrieved_docs = vectorstore.similarity_search(ques, k=5)
  context = "\n\n".join(doc.page_content for doc in retrieved_docs)


  # Prompt template
  system_prompt = (
      "You are a legal assistant answering questions based on Indian criminal law. "
      "If possible, include relevant IPC and BNS section numbers. Be precise and cite sources clearly."
      "Structure the response professionally with clear headings and bullet points where needed."
      "Attach a note at the end that the response is based on the manually created database last updated on 15th Jul 2025, kindly refer to newer sources for any updates."
      "Explain any legal terms in a footnote."
  )

  user_prompt = f"""Question: {ques}

  Context:
  {context}

  Answer:"""

  response = client.chat.completions.create(
      model="meta-llama/llama-4-scout-17b-16e-instruct",
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt}
      ],
      temperature=0.2,
      max_completion_tokens=1024,
      stream=True,
  )

  print("Answer:")
  for chunk in response:
      print(chunk.choices[0].delta.content or "", end="")

